In [1]:
#When to Use RDD as compared to DataFrame with pyspark Job

from pyspark.sql import SparkSession
#INiatilize Spark onject for creating a Dataframe/RDD
spark = SparkSession.builder.appName("RDDvsDataFrame").getOrCreate()

# Create an RDD
rdd = spark.sparkContext.parallelize([(1, "Alice", 30), (2, "Bob", 25), (3, "Charlie", 28)])
print("RDD Data:", rdd.collect())

filtered_rdd = rdd.filter(lambda x: x[2] > 26)
print("Filtered RDD:", filtered_rdd.collect())

25/02/07 09:46:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/02/07 09:46:27 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/02/07 09:46:28 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


RDD Data: [(1, 'Alice', 30), (2, 'Bob', 25), (3, 'Charlie', 28)]


Filtered RDD: [(1, 'Alice', 30), (3, 'Charlie', 28)]


In [2]:
from pyspark.sql import Row

df = spark.createDataFrame(rdd.map(lambda x: Row(ID=x[0], Name=x[1], Age=x[2])))
df.show()


+---+-------+---+
| ID|   Name|Age|
+---+-------+---+
|  1|  Alice| 30|
|  2|    Bob| 25|
|  3|Charlie| 28|
+---+-------+---+



In [3]:
df.filter(df.Age>26).show()

+---+-------+---+
| ID|   Name|Age|
+---+-------+---+
|  1|  Alice| 30|
|  3|Charlie| 28|
+---+-------+---+



In [4]:
# Create an RDD with (ID, Name, Age)
rdd = spark.sparkContext.parallelize([
    (1, "Alice", 30),
    (2, "Bob", 25),
    (3, "Charlie", 28),
    (4, "David", 24),
    (5, "Eve", 32)
])

# Print RDD data
print("RDD Data:", rdd.collect())

# Filter RDD where Age > 26
filtered_rdd = rdd.filter(lambda x: x[2] > 26)
print("Filtered RDD:", filtered_rdd.collect())


RDD Data: [(1, 'Alice', 30), (2, 'Bob', 25), (3, 'Charlie', 28), (4, 'David', 24), (5, 'Eve', 32)]
Filtered RDD: [(1, 'Alice', 30), (3, 'Charlie', 28), (5, 'Eve', 32)]


In [5]:
# Define schema for DataFrame
columns = ["ID", "Name", "Age"]

# Convert RDD to DataFrame
df = rdd.toDF(columns)

# Show DataFrame
df.show()


+---+-------+---+
| ID|   Name|Age|
+---+-------+---+
|  1|  Alice| 30|
|  2|    Bob| 25|
|  3|Charlie| 28|
|  4|  David| 24|
|  5|    Eve| 32|
+---+-------+---+



In [8]:
# DataFrame filter where Age > 26
from pyspark.sql.functions import col
df_filtered = df.filter(col("Age") > 26)
df_filtered.show()



+---+-------+---+
| ID|   Name|Age|
+---+-------+---+
|  1|  Alice| 30|
|  3|Charlie| 28|
|  5|    Eve| 32|
+---+-------+---+



In [9]:
#Aggregation Operations (e.g., avg, max, min, count)
#SQL Queries on DataFrame (SELECT, WHERE, GROUP BY)
#Joins Between DataFrames
#Using Window Functions (rank(), dense_rank(), row_number())

from pyspark.sql.functions import col, avg, min, max, count

# Compute aggregate values
df_agg = df.agg(
    avg("Age").alias("Average_Age"),
    min("Age").alias("Min_Age"),
    max("Age").alias("Max_Age"),
    count("ID").alias("Total_Records")
)

df_agg.show()



+-----------+-------+-------+-------------+
|Average_Age|Min_Age|Max_Age|Total_Records|
+-----------+-------+-------+-------------+
|       27.8|     24|     32|            5|
+-----------+-------+-------+-------------+



In [10]:
# Register DataFrame as SQL Table
df.createOrReplaceTempView("people")

# Run SQL Query: Fetch Names where Age > 26
df_sql = spark.sql("SELECT ID, Name, Age FROM people WHERE Age > 26")
df_sql.show()


+---+-------+---+
| ID|   Name|Age|
+---+-------+---+
|  1|  Alice| 30|
|  3|Charlie| 28|
|  5|    Eve| 32|
+---+-------+---+



In [11]:
from pyspark.sql.functions import lit

# Create another DataFrame for Department
data_dept = [(1, "HR"), (2, "Finance"), (3, "IT"), (4, "HR"), (5, "Finance")]
columns_dept = ["ID", "Department"]

df_dept = spark.createDataFrame(data_dept, columns_dept)

# Perform an INNER JOIN on ID column
df_joined = df.join(df_dept, "ID", "inner")

df_joined.show()


+---+-------+---+----------+
| ID|   Name|Age|Department|
+---+-------+---+----------+
|  5|    Eve| 32|   Finance|
|  1|  Alice| 30|        HR|
|  3|Charlie| 28|        IT|
|  2|    Bob| 25|   Finance|
|  4|  David| 24|        HR|
+---+-------+---+----------+



In [12]:
from pyspark.sql.window import Window
from pyspark.sql.functions import rank, dense_rank, row_number

# Define Window Specification (Ordered by Age Descending)
window_spec = Window.orderBy(col("Age").desc())

# Apply Window Functions
df_window = df.withColumn("Rank", rank().over(window_spec)) \
              .withColumn("Dense_Rank", dense_rank().over(window_spec)) \
              .withColumn("Row_Number", row_number().over(window_spec))

df_window.show()


25/02/07 10:22:34 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+---+-------+---+----+----------+----------+
| ID|   Name|Age|Rank|Dense_Rank|Row_Number|
+---+-------+---+----+----------+----------+
|  5|    Eve| 32|   1|         1|         1|
|  1|  Alice| 30|   2|         2|         2|
|  3|Charlie| 28|   3|         3|         3|
|  2|    Bob| 25|   4|         4|         4|
|  4|  David| 24|   5|         5|         5|
+---+-------+---+----+----------+----------+



In [13]:
#Further Advance operation s are based on 
# handling missing values/ Null Values 
#implementaintg a ETl pipeline
#Performace OPtimization in Pyspark

from pyspark.sql.functions import lit

# Creating DataFrame with NULL values
data_missing = [(1, "Alice", 30), (2, "Bob", None), (3, None, 28), (4, "David", 24), (5, "Eve", None)]
columns_missing = ["ID", "Name", "Age"]

df_missing = spark.createDataFrame(data_missing, columns_missing)

print("Original DataFrame with Missing Values:")
df_missing.show()


Original DataFrame with Missing Values:
+---+-----+----+
| ID| Name| Age|
+---+-----+----+
|  1|Alice|  30|
|  2|  Bob|null|
|  3| null|  28|
|  4|David|  24|
|  5|  Eve|null|
+---+-----+----+



In [14]:
df_dropped = df_missing.dropna()
print("DataFrame after dropping missing values:")
df_dropped.show()


DataFrame after dropping missing values:
+---+-----+---+
| ID| Name|Age|
+---+-----+---+
|  1|Alice| 30|
|  4|David| 24|
+---+-----+---+



In [15]:
df_filled = df_missing.fillna({"Age": 27, "Name": "Unknown"})
print("DataFrame after filling missing values:")
df_filled.show()


DataFrame after filling missing values:
+---+-------+---+
| ID|   Name|Age|
+---+-------+---+
|  1|  Alice| 30|
|  2|    Bob| 27|
|  3|Unknown| 28|
|  4|  David| 24|
|  5|    Eve| 27|
+---+-------+---+



In [16]:
from pyspark.sql.functions import mean

# Calculate mean age
mean_age = df_missing.select(mean(col("Age"))).collect()[0][0]

# Fill missing Age with mean value
df_replaced = df_missing.fillna({"Age": mean_age})
df_replaced.show()


+---+-----+---+
| ID| Name|Age|
+---+-----+---+
|  1|Alice| 30|
|  2|  Bob| 27|
|  3| null| 28|
|  4|David| 24|
|  5|  Eve| 27|
+---+-----+---+

